# First Test for Volatility Based VaR

In [43]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model

# Import functions
import VolatilityVaR

### Data

In [44]:
# Download data
sp500_data = yf.download("^GSPC", start="2017-01-01", end="2021-01-01")
sp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))
returns = sp500_data["Log Returns"].dropna()

[*********************100%***********************]  1 of 1 completed


### Garch based VaR on S&P 500

In [45]:
# Set parameters
confidence_level = 0.99 # <----- Can choose 0.95 etc

In [46]:
# Apply GARCH volatility model
garch_output, next_day_var = VolatilityVaR.var_garch(returns, confidence_level)

print(f"Next-day GARCH VaR estimate (abs): {next_day_var:.2f}%")

# Backtest full period
violations, rate = VolatilityVaR.backtest_var(garch_output, confidence_level)
print(f"Full Period — Violations: {violations}, Rate: {rate:.2f}%")

# Backtest a subset
viol_subset, rate_subset = VolatilityVaR.subset_backtest_var(garch_output, "2019-11-01", "2020-11-30")
print(f"Subset — Violations: {viol_subset}, Rate: {rate_subset:.2f}%")

# Compute ES for the whole period
garch_output = VolatilityVaR.compute_expected_shortfall_volatility(garch_output, confidence_level)

# Plot interactive VaR
fig_var = VolatilityVaR.interactive_plot_var(garch_output)
fig_var.show()

# Plot interactive ES for a subset
fig_es = VolatilityVaR.interactive_plot_es(garch_output, subset=("2019-11-01", "2020-11-30"))
fig_es.show()

# Plot interactive volatility for a subset
fig_vol = VolatilityVaR.interactive_plot_volatility(garch_output["Volatility"], subset=("2019-11-01", "2020-11-30"))
fig_vol.show()


Next-day GARCH VaR estimate (abs): 1.74%
Full Period — Violations: 11, Rate: 1.09%
Subset — Violations: 4, Rate: 1.47%


### MA based VaR on the same data

In [47]:
# Apply MA volatility model
ma_output, next_day_var = VolatilityVaR.var_moving_average(returns, confidence_level)

print(f"Next-day MA VaR estimate (abs): {next_day_var:.2f}%")

# Backtest full period
violations, rate = VolatilityVaR.backtest_var(ma_output, confidence_level)
print(f"Full Period — Violations: {violations}, Rate: {rate:.2f}%")

# Backtest a subset
viol_subset, rate_subset = VolatilityVaR.subset_backtest_var(ma_output, "2019-11-01", "2020-11-30")
print(f"Subset — Violations: {viol_subset}, Rate: {rate_subset:.2f}%")

# Compute ES for the whole period
ma_output = VolatilityVaR.compute_expected_shortfall_volatility(ma_output, confidence_level)

# Plot interactive VaR
fig_var = VolatilityVaR.interactive_plot_var(ma_output)
fig_var.show()

# Plot interactive ES for a subset
fig_es = VolatilityVaR.interactive_plot_es(ma_output, subset=("2019-11-01", "2020-11-30"))
fig_es.show()

# Plot interactive volatility for a subset
fig_vol = VolatilityVaR.interactive_plot_volatility(ma_output["Volatility"], subset=("2019-11-01", "2020-11-30"))
fig_vol.show()


Next-day MA VaR estimate (abs): 1.48%
Full Period — Violations: 10, Rate: 1.01%
Subset — Violations: 4, Rate: 1.47%


### Historical VaR

In [48]:
# Apply Historical VaR model
historical_output, next_day_var = VolatilityVaR.var_historical(returns, confidence_level)

print(f"Historical VaR estimate (abs): {next_day_var:.2f}%")

# Backtest full period
violations, rate = VolatilityVaR.backtest_var(historical_output, confidence_level)
print(f"Full Period — Violations: {violations}, Rate: {rate:.2f}%")

# Compute ES for the whole period
historical_output, es_estimate = VolatilityVaR.compute_es_historical(historical_output, confidence_level)

print(f"Historical ES estimate (abs): {es_estimate:.2f}%")

# Plot interactive VaR
fig_var = VolatilityVaR.interactive_plot_var(historical_output)
fig_var.show()

# Plot interactive ES for a subset
fig_es = VolatilityVaR.interactive_plot_es(historical_output, subset=("2019-11-01", "2020-11-30"))
fig_es.show()


Historical VaR estimate (abs): 3.81%
Full Period — Violations: 11, Rate: 1.09%
Historical ES estimate (abs): 6.23%


### Parametric VaR

In [49]:
# Apply Parametric VaR model (choose distribution: "normal", "t", or "ged")
param_output, next_day_var = VolatilityVaR.var_parametric_iid(returns, confidence_level)

print(f"Parametric VaR estimate (abs): {next_day_var:.2f}%")

# Backtest full period
violations, rate = VolatilityVaR.backtest_var(param_output, confidence_level)
print(f"Full Period — Violations: {violations}, Rate: {rate:.2f}%")

# Compute ES for the whole period
param_output, es_estimate = VolatilityVaR.compute_es_parametric(param_output, returns, confidence_level)

print(f"Parametric ES estimate (abs): {es_estimate:.2f}%")

# Plot interactive VaR
fig_var = VolatilityVaR.interactive_plot_var(param_output)
fig_var.show()

# Plot interactive ES for a subset
fig_es = VolatilityVaR.interactive_plot_es(param_output, subset=("2019-11-01", "2020-11-30"))
fig_es.show()


Parametric VaR estimate (abs): 3.02%
Full Period — Violations: 22, Rate: 2.19%
Parametric ES estimate (abs): 3.46%


### TODO

- Empirical z is correct for ARCH/ GARCH model, but maybe not for EWMA and MA, so assuming a distribution for those may be better. This could explain the very similar violations rate between models.

- Maybe add possibility of using z not empirical but drawn from distributions with volatility models.

- Maybe add different GARCH models (TGARCH etc)

- Maybe add GARCH based simulations (it is complex)

- In the functions, add the possibility to use wealth (W) as input, so it is even more customizable.

- Continue with portfolio methods/ EVT (when Plazzi explains it)